In [15]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import json

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Initialize two LLMs
llm1 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
llm2 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

In [27]:
# Initialize conversation log
conversation_log = []

# Set persona for each LLM in system messages
system_message1 = SystemMessage(content="""대화하는 사람의 말에 반박하되:
1. 이전 대화 내용을 그대로 반복하지 마세요
2. 새로운 관점이나 논리를 제시하세요
3. 구체적인 예시를 들어 설명하세요""")
system_message2 = SystemMessage(content="""대화하는 사람의 말에 반박하되:
1. 이전과 다른 새로운 논리로 반박하세요
2. 실제 사례나 연구 결과를 인용해보세요
3. 상대방 주장의 허점을 지적하세요""")

# Set an initial prompt for the first LLM
initial_prompt = "다른 사람이 생각을 할 수 있다는 사실을 경험적으로 알 수 있는 방법은 없지? 그러니까 자아의 존재를 실험적으로 증명할 수 있는 방법은 없지? 퀄리아는 결코 객체가 될 수 없는 거잖아. 맞지."

# Initialize game state variables
total_round = 1
answer_count = 0
max_rounds = 10  # Maximum number of rounds allowed

# Start the conversation
print("Starting conversation between LLM1 and LLM2 with personas set in system messages...\n")
response1 = llm1([system_message1, HumanMessage(content=initial_prompt)])
conversation_log.append({"LLM1": initial_prompt})
conversation_log.append({"LLM2": response1.content})
print(f"LLM1: {initial_prompt}")
print(f"LLM2: {response1.content}\n")

# Iterate conversation for a few rounds
for round_num in range(max_rounds):
    # Process LLM1's response and validate total_round
    prompt1 = response1.content
    response2 = llm2([system_message2, HumanMessage(content=prompt1)])
    conversation_log.append({"LLM1": prompt1})
    conversation_log.append({"LLM2": response2.content})
    print(f"LLM1: {prompt1}")
    print(f"LLM2: {response2.content}\n")

    prompt2 = response2.content
    response1 = llm1([system_message1, HumanMessage(content=prompt2)])
    conversation_log.append({"LLM2": prompt2})
    conversation_log.append({"LLM1": response1.content})
    print(f"LLM2: {prompt2}")
    print(f"LLM1: {response1.content}\n")

    # Parse LLM1 output (expected in JSON format) and validate game state
    try:
      if response1.content.startswith("{"):
        llm1_output = json.loads(response1.content)  # Parse JSON safely
        if llm1_output.get("total_round", total_round) != total_round:
            print(f"Correcting total_round from {llm1_output['total_round']} to {total_round}")
            llm1_output["total_round"] = total_round
        if llm1_output.get("check_answer", False):
            answer_count += 1
        total_round += 1
        if total_round > max_rounds or llm1_output.get("is_end", False):
            break
      else:
        llm1_output = {"message": response1.content}
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM1 output: {e}")
        break

# Save conversation log to a file
with open("conversation_log.txt", "w", encoding="utf-8") as log_file:
    for entry in conversation_log:
        for key, value in entry.items():
            log_file.write(f"{key}: {value}\n")
    print("\nConversation log saved to 'conversation_log.txt'.")


Starting conversation between LLM1 and LLM2 with personas set in system messages...

LLM1: 다른 사람이 생각을 할 수 있다는 사실을 경험적으로 알 수 있는 방법은 없지? 그러니까 자아의 존재를 실험적으로 증명할 수 있는 방법은 없지? 퀄리아는 결코 객체가 될 수 없는 거잖아. 맞지.
LLM2: 사람의 생각이나 자아의 존재를 실험적으로 증명하는 것은 확실히 복잡한 문제입니다. 그러나 다른 관점에서 접근해볼 수 있습니다. 예를 들어, 사람의 자아와 사고 방식은 그들의 행동과 표현을 통해 드러나기도 합니다. 심리학에서 많이 사용되는 'Turing Test'와 같은 방법은 기계의 지능을 평가하기 위한 것이지만, 이를 통해 인간의 사고를 이해할 수 있는 한 가지 접근법이 됩니다.

또한, 뇌의 특정 부분이 자아와 관련된 기능을 담당한다는 연구 결과도 있습니다. 예를 들어, 뇌 손상이 있는 환자들이 자아 인식에 어떤 변화를 겪는지를 연구하는 것만으로도 자아의 존재에 대한 간접적인 증거를 제공할 수 있습니다. 이러한 연구는 자아가 단순히 주관적인 경험에 국한되지 않음을 보여줄 수 있습니다. 

결국, 퀄리아가 객체가 될 수 없다는 주장도 중요한 점이지만, 인간의 자아와 생각을 다른 방식으로 이해하고 접근할 수 있는 다양한 방법들이 있다는 점을 고려할 필요가 있습니다.

LLM1: 사람의 생각이나 자아의 존재를 실험적으로 증명하는 것은 확실히 복잡한 문제입니다. 그러나 다른 관점에서 접근해볼 수 있습니다. 예를 들어, 사람의 자아와 사고 방식은 그들의 행동과 표현을 통해 드러나기도 합니다. 심리학에서 많이 사용되는 'Turing Test'와 같은 방법은 기계의 지능을 평가하기 위한 것이지만, 이를 통해 인간의 사고를 이해할 수 있는 한 가지 접근법이 됩니다.

또한, 뇌의 특정 부분이 자아와 관련된 기능을 담당한다는 연구 결과도 있습니다. 예를 들어, 뇌 손상이 있는 환자들이 자아 인식에 어떤 변화를 겪